<a href="https://colab.research.google.com/github/shahzad-r1zv1/Python_Course/blob/main/Week_5_Advance_Topics_in_Chatbots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ====================================
# Week 5: Advance Topics in Chatbots
# ====================================



#Goal
Build a chatbot that uses GPT-style models to assist software testers and QA engineers with:

*   Test case generation
*   Writing PyTest or Robot Framework code
*   Explaining testing concepts
*   Suggesting test data or edge cases
*   Answering common QA interview questions
*   All built in Google Colab, optionally with a Gradio UI.


---


#Tools & Libraries:
 - transformers library (Hugging Face)
 - Pre-trained model: gpt2 or flan-t5-base
 - Optionally: gradio for UI, datasets for finetuning (advanced)

---


#Lesson Breakdown
## Chatbot? Let's make it more intellegent!
**Key Learning Objective:** Use GenerativeLLM to build a more complex example.



In [ ]:
# Step 1: Install Dependencies
!pip install transformers gradio

### Task 1:

**Topics:**


In [ ]:
# Step 2: Import Libraries and Initialize Models
from transformers import pipeline, set_seed
from difflib import SequenceMatcher

set_seed(42)  # for reproducibility

# GPT-2 Model for text-generation
gpt2_bot = pipeline("text-generation", model="gpt2")

# FLAN-T5 Model for text-to-text-generation (instruction following)
flan_t5_bot = pipeline("text2text-generation", model="google/flan-t5-base")

### Task 2:


In [ ]:
# Step 3: Define a Similarity Function to Compare Two Strings
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Step 4: Define the Ensemble Chatbot Function
def generate_response(user_input):
    # Prepare prompts for each model
    prompt_gpt = f"You are a helpful software QA assistant. Answer this: {user_input}\n\n"
    prompt_fl = f"Help a QA tester: {user_input}"

    # Generate responses from both models
    response1 = gpt2_bot(prompt_gpt, max_length=256, do_sample=True, temperature=0.7, top_p=0.9)[0]["generated_text"]
    response1 = response1.replace(prompt_gpt, "").strip()

    response2 = flan_t5_bot(prompt_fl, max_length=256)[0]["generated_text"].strip()

    # Compare the two responses using a similarity score
    sim = similarity(response1, response2)

    if sim > 0.8:
        # The responses are very similar—return one of them.
        optimal_response = response1
    else:
        # The responses differ—combine them for a richer output.
        optimal_response = f"{response1}\n\nAdditionally:\n{response2}"

    return optimal_response


### Task 3:

Topics:

-

In [ ]:
# Step 5: Test the Ensemble Response Function Interactively
user_input = input("Enter your QA question: ")
print("Bot:", generate_response(user_input))




### Task 4:
Topics:
- **Chatbots?**

In [ ]:
# Step 6: Launch a Gradio Interface for the Chatbot
import gradio as gr

gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Ask a QA question, e.g. 'Write a PyTest test for login'"),
    outputs="text",
    title="QA Assistant Chatbot Ensemble",
    description="This chatbot uses a combination of GPT-2 and FLAN-T5 to generate optimal responses for software testers and QA engineers."
).launch()


# ===========
# What's Next?
# ===========

1. Apply NLP & AI Concepts
Implement text processing & sentiment analysis in your chatbot/project.
Improve chatbot responses with custom datasets or external APIs.
2. Define User Interactions
How will users input queries? (Text-based, voice input, button clicks, etc.)
How will the bot respond intelligently? (Rule-based, AI-powered, dynamic responses).
3. Work on API Integrations
If applicable, integrate weather APIs, news APIs, or a database.
4. Implement Basic Test Cases
Start writing PyTest cases to validate chatbot behavior.





# ===========
# Weekly Summary
# ===========
- Learned NLP Basics & AI Chatbots
- Preprocessed Text Data (Tokenization, Lemmatization, Stopword Removal)
- Built a Sentiment Analyzer
- Implemented a Chatbot with ChatterBot
- Integrated APIs to Enhance Functionality